## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import optuna
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import lightgbm as lgb

### Data Load

In [2]:
# 데이터 로드
Total_train = pd.read_csv('../data/Total_train_dataset_32.csv')
Total_test = pd.read_csv('../data/Total_test_dataset_32.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']

### 인코딩 

In [4]:
Total_categorical_columns = [
    "시술_당시_나이",
    "배란_유도_유형",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이"
]

In [5]:
# 모든 범주형 변수를 문자열로 변환
Total_X[Total_categorical_columns] = Total_X[Total_categorical_columns].astype(str)
Total_test[Total_categorical_columns] = Total_test[Total_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
Total_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

Total_X[Total_categorical_columns] = Total_encoder.fit_transform(Total_X[Total_categorical_columns])
Total_test[Total_categorical_columns] = Total_encoder.transform(Total_test[Total_categorical_columns])

## Modeling

In [ ]:
# 데이터 분할
Total_X_train, Total_X_test, Total_y_train, Total_y_test = train_test_split(Total_X, Total_y, test_size=0.2, random_state=42)

: 

### Total 데이터

optuna

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score

# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 4000)
        , 'max_depth': trial.suggest_int('max_depth', 20, 80)
        , 'min_samples_split': trial.suggest_int('min_samples_split', 2, 16)
        , 'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 8)
        , 'criterion': trial.suggest_categorical('criterion', ['entropy'])
        , 'bootstrap': trial.suggest_categorical('bootstrap', [False])
        
        , 'random_state': 42
        , 'n_jobs': -1
    }

    model = ExtraTreesClassifier(**param)
    model.fit(Total_X_train, Total_y_train)
    
    y_pred_proba = model.predict_proba(Total_X_test)[:, 1]
    
    auc = roc_auc_score(Total_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-17 18:41:26,984] A new study created in memory with name: no-name-3df18429-d204-40da-8108-ec2310c84877
[I 2025-02-17 18:52:35,416] Trial 0 finished with value: 0.7296051149898266 and parameters: {'n_estimators': 1994, 'max_depth': 60, 'min_samples_split': 12, 'min_samples_leaf': 2, 'criterion': 'entropy', 'bootstrap': False}. Best is trial 0 with value: 0.7296051149898266.
[I 2025-02-17 19:11:07,476] Trial 1 finished with value: 0.7338994020109713 and parameters: {'n_estimators': 3737, 'max_depth': 33, 'min_samples_split': 16, 'min_samples_leaf': 5, 'criterion': 'entropy', 'bootstrap': False}. Best is trial 1 with value: 0.7338994020109713.
[I 2025-02-17 19:19:09,165] Trial 2 finished with value: 0.7348343819313456 and parameters: {'n_estimators': 1622, 'max_depth': 73, 'min_samples_split': 4, 'min_samples_leaf': 8, 'criterion': 'entropy', 'bootstrap': False}. Best is trial 2 with value: 0.7348343819313456.
[I 2025-02-17 19:41:08,014] Trial 3 finished with value: 0.730778879

.